In [57]:
import struct

class SHA1:
    def __init__(self, data):
        self.data = data
        self.ascii2bin()
        self.h = [
            0x67452301,
            0xEFCDAB89,
            0x98BADCFE,
            0x10325476,
            0xC3D2E1F0
        ]
        self.final_hash()

    @staticmethod
    def left_rotate(n, b):
        return ((n << b) | (n >> (32 - b))) & 0xFFFFFFFF

    def ascii2bin(self):
        self.data = ''.join(format(x, 'b').zfill(8) for x in self.data)

    def padding(self):
        length = len(self.data)
        self.data += "1"

        fillValue = (len(self.data) // 512) + 1
        fillValue = (fillValue * 512) - 64

        if len(self.data) % 512 != 448:
            self.data = self.data.ljust(fillValue, '0')

        bitLength = bin(length)[2:]
        bitLength = bitLength.zfill(64)

        self.data += bitLength

    def split2chunks(self):
        self.chunks = [self.data[i: i + 512] for i in range(0, len(self.data), 512)]

    def chunks2words(self, chunk):
        return [chunk[i: i + 32] for i in range(0, 512, 32)]

    def extend_words(self, words):
        extendedWords = [0] * 80
        # Copying data from words as int
        for i in range(0, len(words)):
            extendedWords[i] = int(words[i], 2)
            
        for i in range(16, 80):
            temp = self.left_rotate(
                (extendedWords[i-3] ^ extendedWords[i-8] ^ extendedWords[i-14] ^ extendedWords[i-16]), 1)
            extendedWords[i] = temp

        return extendedWords

    def final_hash(self):
        self.padding()
        self.split2chunks()

        for chunk in self.chunks:
            words = self.chunks2words(chunk)
            extendedWords = self.extend_words(words)
            a, b, c, d, e = self.h

            for i in range(0, len(extendedWords)):
                if 0 <= i <= 19:
                    f = d ^ (b & (c ^ d))
                    k = 0x5A827999

                elif 20 <= i <= 39:
                    f = b ^ c ^ d
                    k = 0x6ED9EBA1

                elif 40 <= i <= 59:
                    f = (b & c) | (b & d) | (c & d)
                    k = 0x8F1BBCDC

                elif 60 <= i <= 79:
                    f = b ^ c ^ d
                    k = 0xCA62C1D6

                temp = (self.left_rotate(a, 5) + f + e + k + extendedWords[i]) & 0xFFFFFFFF
                e = d
                d = c
                c = self.left_rotate(b, 30)
                b = a
                a = temp

            self.h = (
                self.h[0] + a & 0xFFFFFFFF,
                self.h[1] + b & 0xFFFFFFFF,
                self.h[2] + c & 0xFFFFFFFF,
                self.h[3] + d & 0xFFFFFFFF,
                self.h[4] + e & 0xFFFFFFFF)
        return self.hexdigest()

    def hexdigest(self):      
        sha1Digest = hex(self.h[0])[2:].zfill(8) + hex(self.h[1])[2:].zfill(8) + hex(self.h[2])[2:].zfill(8) + hex(self.h[3])[2:].zfill(8) + hex(self.h[4])[2:].zfill(8)
        return sha1Digest


def execute():
    userInput = input("Enter the string to hash > ")
    userInput = bytes(userInput, 'utf-8')
    hashObject = SHA1(userInput)
    print("SHA-1 digest: " + hashObject.hexdigest())
    


In [58]:
class HMAC:
    def __init__(self, key, message, hash_h=SHA1):
        self.i_key_pad = bytearray()
        self.o_key_pad = bytearray()
        self.key = key
        self.message = message
        self.blocksize = 64
        self.hash_h = hash_h
        self.init_flag = False
        
    def init_pads(self):
        for i in range(self.blocksize):
            self.i_key_pad.append(0x36 ^ self.key[i])
            self.o_key_pad.append(0x5C ^ self.key[i])
        
    def init_key(self):
        if len(self.key) > self.blocksize:
            self.key = bytearray(SHA1(key).final_hash())
        elif len(self.key) < self.blocksize:
            i = len(self.key)
            while i < self.blocksize:
                self.key += b"\x00"
                i += 1
        
    def digest(self):
        if self.init_flag == False:
            self.init_key()
            self.init_pads()
            self.init_flag = True
        return self.hash_h(bytes(self.o_key_pad) + self.hash_h(bytes(self.i_key_pad) + self.message).digest()).digest()
    
    def hexdigest(self):
        if self.init_flag == False:
            self.init_key()
            self.init_pads()
            self.init_flag = True

        return self.hash_h(bytes(self.o_key_pad) + bytes(self.hash_h(bytes(self.i_key_pad) + self.message).final_hash(),"utf-8")).final_hash()

h = HMAC(b"adhil", b"crypto lab", SHA1)
key = "adhil"
message = "crypto lab"
print("Key:" + key)
print("Message:" + message)
output = h.hexdigest()
print("Output Digest:" + output)


Key:adhil
Message:crypto lab
Output Digest:6eae47399b8a78e1c093c47e3892f672f5e63cd2
